## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [1]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')
df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [3]:
# View the column names in the data
df.columns

Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

In [4]:
# Use the describe function to gather some basic statistics
df.describe()

,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.000000,5.463900e+04,54639.000000,54639.000000,54639.000000,54639.000000,54639.000000,5.463900e+04,54639.000000
mean,54837.869416,5.470190e+06,11.359139,2022.993064,136.267207,99.446073,5.004116,5.702646e+02,2.979667
std,25487.438231,2.599807e+06,7.023499,0.082997,183.873135,133.164267,5.326599,1.879552e+04,2.436320
min,10033.000000,1.000886e+06,1.000000,2022.000000,0.010000,0.010000,0.000000,0.000000e+00,0.000000
25%,33593.000000,3.196372e+06,6.000000,2023.000000,20.800000,14.840000,1.450000,3.200000e+01,1.000000
50%,53305.000000,5.496966e+06,11.000000,2023.000000,68.310000,49.890000,3.240000,6.800000e+01,3.000000
75%,78498.000000,7.733869e+06,17.000000,2023.000000,173.160000,125.570000,6.890000,1.700000e+02,5.000000
max,99984.000000,9.998480e+06,52.000000,2023.000000,1396.230000,846.270000,46.430000,3.958244e+06,9.000000


In [9]:
# Use this space to do any additional research
# and familiarize yourself with the data.

#Average unit price per category
avg_unit_price_per_category = df.groupby('category')['unit_price'].mean()

avg_unit_price_per_category

category
consumables    128.232610
decor          165.913406
electronics    156.201342
furniture      126.052812
software       138.477150
Name: unit_price, dtype: float64

In [11]:
#Total orders per year
orders_per_year = df['order_year'].value_counts().sort_index()
orders_per_year

order_year
2022      379
2023    54260
Name: count, dtype: int64

In [2]:
# What three item categories had the most entries?
top_categories = df['category'].value_counts().head(3)
top_categories

category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64

In [3]:
# For the category with the most entries, which subcategory had the most entries?
top_category = top_categories.idxmax()
top_subcategory = df[df['category'] == top_category]['subcategory'].value_counts().idxmax()
top_subcategory

'bathroom supplies'

In [4]:
# Which five clients had the most entries in the data?
top_clients = df['client_id'].value_counts().head(5)
top_clients

client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64

In [6]:
# Store the client ids of those top 5 clients in a list.
top_client_ids = top_clients.index.tolist()
top_client_ids

[33615, 66037, 46820, 38378, 24741]

In [7]:
# How many total units (the qty column) did the client with the most entries order order?
top_client_total_units = df[df['client_id'] == top_client_ids[0]]['qty'].sum()
top_client_total_units

64313

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [12]:
# Create a column that calculates the subtotal for each line using the unit_price and the qty
df['subtotal'] = df['unit_price'] * df['qty']

In [13]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound for orders over 50 pounds and $10 per pound for items 50 pounds or under.
def calculate_shipping_price(weight):
    return 7 * weight if weight > 50 else 10 * weight

df['shipping_price'] = df['unit_weight'] * df['qty'].apply(calculate_shipping_price)


In [14]:
# Create a column for the total price using the subtotal and the shipping price along with a sales tax of 9.25%
df['total_price'] = (df['subtotal'] + df['shipping_price']) * 1.0925


In [15]:
# Create a column for the cost of each line using unit cost, qty, and
# shipping price (assume the shipping cost is exactly what is charged to the client).
df['line_cost'] = (df['unit_cost'] * df['qty']) + df['shipping_price']



In [17]:
# Create a column for the profit of each line using line cost and line price
df['profit'] = df['total_price'] - df['line_cost']

df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_price,unit_cost,unit_weight,qty,line_number,subtotal,shipping_price,total_price,line_cost,profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,1096.80,762.71,7.50,105,1,115164.00,5512.5,131839.076250,85597.05,46242.026250
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,24.95,15.09,1.49,21,0,523.95,312.9,914.258625,629.79,284.468625
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,13.52,7.86,1.68,39,6,527.28,655.2,1291.859400,961.74,330.119400
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,36.42,24.85,1.23,29,3,1056.18,356.7,1543.571400,1077.35,466.221400
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,195.10,108.17,46.43,20,1,3902.00,9286.0,14407.890000,11449.40,2958.490000


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [19]:
# Check your work using the totals above
# Define the order IDs and their expected total prices
# Part 3: Confirm Your Work

# Define the order IDs and their expected total prices
order_totals = {
    2742071: 152811.89,
    2173913: 162388.71,
    6128929: 923441.25
}

# Calculate the total prices for each order ID in the dataset
calculated_totals = df.groupby('order_id')['total_price'].sum()

# Function to print and check each order total
def check_order_total(order_id, expected_total):
    order_df = df[df['order_id'] == order_id]
    calculated_total = order_df['total_price'].sum()
    print(f"Order ID {order_id} Details:\n{order_df}")
    print(f"Calculated Total = ${calculated_total:.2f}, Expected Total = ${expected_total:.2f}")
    assert abs(calculated_total - expected_total) < 1e-2, f"Total for order ID {order_id} does not match the expected total!"

# Verify the results
for order_id, expected_total in order_totals.items():
    check_order_total(order_id, expected_total)

# If the assertions pass, print a confirmation message
print("All order totals match the provided receipts.")


Order ID 2742071 Details:
          first   last                                       job  \
3446   Kimberly  Blair  Geographical information systems officer   
6287   Kimberly  Blair  Geographical information systems officer   
20623  Kimberly  Blair  Geographical information systems officer   
23840  Kimberly  Blair  Geographical information systems officer   
24671  Kimberly  Blair  Geographical information systems officer   
42470  Kimberly  Blair  Geographical information systems officer   

                    phone                           email  client_id  \
3446   477-680-9323x27378  blair.kimberly.6618@nelson.biz      78447   
6287   477-680-9323x27378  blair.kimberly.6618@nelson.biz      78447   
20623  477-680-9323x27378  blair.kimberly.6618@nelson.biz      78447   
23840  477-680-9323x27378  blair.kimberly.6618@nelson.biz      78447   
24671  477-680-9323x27378  blair.kimberly.6618@nelson.biz      78447   
42470  477-680-9323x27378  blair.kimberly.6618@nelson.biz      78

AssertionError: Total for order ID 2742071 does not match the expected total!

## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [16]:
# How much did each of the top 5 clients by quantity spend? Check your work from Part 1 for client ids.
top_clients_spent = df[df['client_id'].isin(top_client_ids)].groupby('client_id')['total_price'].sum()



33615: $8377308.52
66037: $10259514.79
46820: $9743794.36
38378: $12906550.87
24741: $82268892.02


In [20]:
# Create a summary DataFrame showing the totals for the for the top 5 clients with the following information:
# total units purchased, total shipping price, total revenue, and total profit. 

summary_df = df[df['client_id'].isin(top_client_ids)].groupby('client_id').agg({
    'qty': 'sum',
    'shipping_price': 'sum',
    'total_price': 'sum',
    'profit': 'sum'
}).reset_index()

In [26]:
# Format the data and rename the columns to names suitable for presentation.
summary_df.columns = ['Client ID', 'Total Units Purchased', 'Total Shipping Price', 'Total Revenue', 'Total Profit']


# Function to convert currency to millions of dollars
def to_millions(value):
    return value / 1_000_000
# Define the money columns. 


# Define a function that converts a dollar amount to millions.


# Apply the currency_format_millions function to only the money columns. 


# Rename the columns to reflect the change in the money format. 


# Apply the function to the relevant columns
summary_df['Total Shipping Price'] = summary_df['Total Shipping Price'].apply(to_millions)
summary_df['Total Revenue'] = summary_df['Total Revenue'].apply(to_millions)
summary_df['Total Profit'] = summary_df['Total Profit'].apply(to_millions)

# Rename columns for presentation
summary_df.columns = ['Client ID', 'Total Units Purchased', 'Total Shipping Price (in millions)', 'Total Revenue (in millions)', 'Total Profit (in millions)']


In [27]:
# Sort the updated data by "Total Profit (millions)" form highest to lowest and assign the sort to a new DatFrame.
summary_df = summary_df.sort_values(by='Total Profit (in millions)', ascending=False)

# Display the summary DataFrame
summary_df

,Client ID,Total Units Purchased,Total Shipping Price (in millions),Total Revenue (in millions),Total Profit (in millions)
0,24741,239862,0.000005,0.000082,0.000037
2,38378,73667,0.000003,0.000013,0.000003
4,66037,43018,0.000001,0.000010,0.000003
3,46820,75768,0.000002,0.000010,0.000003
1,33615,64313,0.000002,0.000008,0.000002


In [28]:
summary_text = """
The analysis shows that the top 5 clients by quantity have significantly contributed to the company's revenue and profit. The Total Profit for these clients ranges from X to Y million dollars. The highest spending client ordered over Z units and generated the most profit.
"""

print(summary_text)


The analysis shows that the top 5 clients by quantity have significantly contributed to the company's revenue and profit. The Total Profit for these clients ranges from X to Y million dollars. The highest spending client ordered over Z units and generated the most profit.

